# Producing Data Understanding Reports in Markdown


## Import the report generator module


In [10]:
import sys
sys.path.append('../') 

from hnm_data_analysis.data_understanding.data_report_generator import generate_data_report

## Generate Reports


In [11]:
try:
    # Generate data report for customers.csv
    generate_data_report("../data/raw/customers.csv")

    # Generate data report for transactions.csv with a sample size of 10000
    generate_data_report("../data/raw/transactions_train.csv", sample_size=10000)

    # Generate data report for articles.csv
    generate_data_report("../data/raw/articles.csv")

except Exception as e:
    print(f"Error: {e}")